In [6]:
import numpy as np


def remove_txt_headers(path):
    
    no_lines = ['Interval', 'ExcelDateTime', 'TimeFormat', 'DateFormat', 'ChannelTitle', 'Range', 'UnitName', 'TopValue', 
                'BottomValue']  # Lines to remove
    
    # Read raw lines
    with open(f'raw_labchart/{path}', 'r') as f:
        lines = f.readlines()
    
    # Remove lines
    lines_arr = np.array(lines)
    lines_arr = lines_arr[[not any(x in y for x in no_lines) for y in lines_arr]]

    # Write new file
    s = ''.join(lines_arr)
    with open('headers_removed/{0}_out.txt'.format(path.split('.')[0]), 'w') as f:
        f.write(s)

In [8]:
remove_txt_headers('worm4.txt')

In [10]:
import pandas as pd
df = pd.read_excel('excel_data/worm1.xlsx')
df.columns

Index(['time', 'stimulus', 'signal', 'comments'], dtype='object')